<a href="https://colab.research.google.com/github/ChabiMax/author_network/blob/main/Scraping%20propre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from tqdm import *
import time
import requests
import bs4
from collections import Counter
import json
import os
from google.colab import files
import pickle as pickle
import pandas as pd
import networkx as nx

raw_data = {}

# Scraping

In [7]:
# Extraction de la liste d'auteurs

URL = 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary = {}
date_naissance = None
date_mort = None
name = 'init'
author_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    if not any(char.isdigit() for char in string) :
      if serie == 2 :
        serie = 0
        date_mort = 2025
        dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}
      serie = serie+1
      id = id+1
      name = string
      date_naissance = None
      date_mort = None
      if 'https://' in link.get('href'):
        author_link = link.get('href')
      else:
        author_link = 'https://fr.wikipedia.org' + link.get('href')
    else :
      if serie == 1 :
        serie = serie+1
        date_naissance = string
      else :
        if serie == 2 :
          serie = 0
          date_mort = string
          dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}

# Nettoyage manuel
unwanted_list = list(range(0,5))
unwanted_list += list(range(1793,1806))
unwanted_list += [key for key in dictionary.keys() if 'modifier' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'siècle' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'ms.' in dictionary[key]['name']]

dictionary = dict((key, dictionary[key]) for key in dictionary.keys() if key not in unwanted_list)

In [8]:
 # Extraction de la liste des genres littéraires

URL = 'https://fr.wikipedia.org/wiki/Genre_litt%C3%A9raire'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary_genres = {}
name = 'init'
genre_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    id = id+1
    name = string
    if 'https://' in link.get('href'):
      genre_link = link.get('href')
    else:
      genre_link = 'https://fr.wikipedia.org' + link.get('href')
    dictionary_genres[id] = {'name' : name, 'link' : genre_link}

# Nettoyage manuel
unwanted_list = list(range(0,49))
unwanted_list += list(range(278,344))
unwanted_list += [key for key in dictionary_genres.keys() if 'modifier' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if dictionary_genres[key]['link'] in [dictionary[key_bis]['link'] for key_bis in dictionary.keys()]]
unwanted_list += [key for key in dictionary_genres.keys() if any(char.isdigit() for char in dictionary_genres[key]['name'])]
unwanted_list += list(range(285,len(dictionary_genres.keys())))
dictionary_genres = dict((key, dictionary_genres[key]) for key in dictionary_genres.keys() if key not in unwanted_list)


In [33]:
# Reset des keys 
old_dictionary = dictionary.copy()
dictionary = {}
id = 0
for key in old_dictionary :
  dictionary[id] = old_dictionary[key]
  id += 1

old_dictionary_genres = dictionary_genres.copy()
dictionary_genres = {}
for key in old_dictionary_genres :
  dictionary_genres[id] = old_dictionary_genres[key]
  id += 1

In [34]:
# Parcours des pages et création du réseau
author_links = [dictionary[key]['link'] for key in dictionary.keys()]
genre_links = [dictionary_genres[key]['link'] for key in dictionary_genres.keys()]

for id, values in tqdm_notebook(dictionary.items()) :
  link = values['link']
  response = requests.get(link)
  soup = bs4.BeautifulSoup(response.text, 'html.parser')
  links = soup.find(id="bodyContent").select('a')
  network_aslink = []
  net_genre_aslink = []
  for link in links:
    if link.get('href') != None:
      if 'https://' in link.get('href'):
        link = link.get('href')
      else:
        link = 'https://fr.wikipedia.org' + link.get('href')
    if link in author_links :
      network_aslink.append(link)
    if link in genre_links :
      net_genre_aslink.append(link)
  network = []
  net_genre = []
  for link in network_aslink :
    for id_bis, values_bis in dictionary.items() :
      if link == values_bis['link'] :
        network.append(id_bis)
  for link in net_genre_aslink :
    for id_bis, values_bis in dictionary_genres.items() :
      if link == values_bis['link'] :
        net_genre.append(id_bis)
  counted_network = Counter(network)
  counted_net_genre = Counter(net_genre)
  dictionary[id]['taille_html_article'] = len(str(soup))
  dictionary[id]['network_interauteur'] = list(counted_network.keys())
  dictionary[id]['poids_liens_interauteur'] = list(counted_network.values())
  dictionary[id]['genres_mentionnes'] = list(counted_net_genre.keys())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1583 [00:00<?, ?it/s]

# Formatage données

In [49]:
raw_data1 = dictionary.copy()
for key in raw_data1.keys() :
  raw_data1[key]['type'] = 'auteur'

for key in dictionary_genres.keys() :
  raw_data1[key] = dictionary_genres[key].copy()
  raw_data1[key]['type'] = 'genre'

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['auteurs_du_genre'] = []
for key_auteur in raw_data1.keys() :
  if raw_data1[key_auteur]['type'] == 'auteur' :
    for genre in raw_data1[key_auteur]['genres_mentionnes'] :
      raw_data1[genre]['auteurs_du_genre'] = raw_data1[genre]['auteurs_du_genre'] + [key_auteur]

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'auteur' :
    try :
      raw_data1[key_genre]['date_naissance'] = int(raw_data1[key_genre]['date_naissance'])
    except :
      raw_data1[key_genre]['date_naissance'] = 9999
    try :
      raw_data1[key_genre]['date_mort'] = int(raw_data1[key_genre]['date_mort'])
    except :
      raw_data1[key_genre]['date_mort'] = 9999

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['debut_genre'] = min([raw_data1[auteur]['date_naissance'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur'] + [0])
    raw_data1[key_genre]['fin_genre'] = max([raw_data1[auteur]['date_mort'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur'] + [9999])

for key_auteur1 in raw_data1.keys() :
  if raw_data1[key_auteur1]['type'] == 'auteur' :
    for key_auteur2 in raw_data1.keys() :
      if (raw_data1[key_auteur2]['type'] == 'auteur') & (key_auteur2 != key_auteur1) :
        pass

# mort_1 = [raw_data1[key_auteur1]['date_mort'] if raw_data1[key_auteur1]['date_mort'] != None else 9999]
# mort_2 = [raw_data1[key_auteur2]['date_mort'] if raw_data1[key_auteur2]['date_mort'] != None else 9999]
# naissance_1 = [raw_data1[key_auteur1]['date_naissance'] if raw_data1[id]['date_mort'] != None else 0]
# naissance_2 = [raw_data1[key_auteur2]['date_naissance'] if raw_data1[key_auteur2]['date_mort'] != None else 0]
# try:
#   if (mort_1 > naissance_2 and naissance_1 < mort_2) or (mort_2 > naissance_1 and naissance_2 < mort_1):
#     raw_data1[key_auteur1]['contemporain'].append(key_auteur2)
#     raw_data1[key_auteur2]['contemporain'].append(key_auteur1)
# except :
#   pass


In [50]:
raw_data1

{0: {'date_mort': 2025,
  'date_naissance': 9999,
  'genres_mentionnes': [],
  'link': 'https://fr.wikipedia.org/wiki/Turold',
  'name': 'Turold',
  'network_interauteur': [1582],
  'poids_liens_interauteur': [1],
  'taille_html_article': 72758,
  'type': 'auteur'},
 1: {'date_mort': 2025,
  'date_naissance': 9999,
  'genres_mentionnes': [],
  'link': 'https://fr.wikipedia.org/wiki/Philippe_de_Thaon',
  'name': 'Philippe de Thaon',
  'network_interauteur': [1582],
  'poids_liens_interauteur': [1],
  'taille_html_article': 67953,
  'type': 'auteur'},
 2: {'date_mort': 2025,
  'date_naissance': 9999,
  'genres_mentionnes': [1626],
  'link': 'https://fr.wikipedia.org/wiki/Beno%C3%AEt_de_Sainte-Maure',
  'name': 'Benoît de Sainte-Maure',
  'network_interauteur': [4, 1582],
  'poids_liens_interauteur': [1, 1],
  'taille_html_article': 82458,
  'type': 'auteur'},
 3: {'date_mort': 1165,
  'date_naissance': 9999,
  'genres_mentionnes': [1661, 1690, 1603],
  'link': 'https://fr.wikipedia.org/w

In [ ]:
genre

In [ ]:
raw_data1[1

In [ ]:
raw_data1[20]